# 03. Conventional RAG based on LangChain

Q: Retireval을 더 잘하게 수정해보자!

In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_teddynote import logging

logging.langsmith(project_name="Paper-Agent")

LangSmith 추적을 시작합니다.
[프로젝트명]
Paper-Agent


## 여러 PDF 문서 로드 비교

참고: https://python.langchain.com/docs/integrations/document_loaders/#pdfs


- PyPDF
- PDFPlumber
- PyPDFium2
- PyMuPDF
- PDFMiner

In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_community.document_loaders import PyPDFium2Loader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import PDFMinerLoader

In [5]:
loader = PyPDFLoader(
    "../data/Retrieval Augmented Generation for Knowledge Intensive NLP Tasks.pdf"
)
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")
print(docs[0].page_content[:300])

문서의 페이지수: 19
Retrieval-Augmented Generation for
Knowledge-Intensive NLP Tasks
Patrick Lewis†‡, Ethan Perez⋆,
Aleksandra Piktus†, Fabio Petroni†, Vladimir Karpukhin†, Naman Goyal†, Heinrich Küttler†,
Mike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, Sebastian Riedel†‡, Douwe Kiela†
†Facebook AI Research; ‡University 


In [6]:
loader = PDFPlumberLoader(
    "../data/Retrieval Augmented Generation for Knowledge Intensive NLP Tasks.pdf"
)
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")
print(docs[0].page_content[:300])

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


문서의 페이지수: 19
Retrieval-Augmented Generation for
Knowledge-Intensive NLP Tasks
PatrickLewis†‡,EthanPerez(cid:63),
AleksandraPiktus†,FabioPetroni†,VladimirKarpukhin†,NamanGoyal†,HeinrichKüttler†,
MikeLewis†,Wen-tauYih†,TimRocktäschel†‡,SebastianRiedel†‡,DouweKiela†
†FacebookAIResearch;‡UniversityCollegeLondon;(cid


In [7]:
loader = PyPDFium2Loader(
    "../data/Retrieval Augmented Generation for Knowledge Intensive NLP Tasks.pdf"
)
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")
print(docs[0].page_content[:300])

문서의 페이지수: 19
Retrieval-Augmented Generation for
Knowledge-Intensive NLP Tasks
Patrick Lewis†‡, Ethan Perez?,
Aleksandra Piktus†, Fabio Petroni†, Vladimir Karpukhin†, Naman Goyal†, Heinrich Küttler†,
Mike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, Sebastian Riedel†‡, Douwe Kiela†
†Facebook AI Research; ‡University 


/Users/sean/lagom/paper-study-agent/.venv/lib/python3.11/site-packages/pypdfium2/_helpers/textpage.py:80: UserWarning: get_text_range() call with default params will be implicitly redirected to get_text_bounded()
  warnings.warn("get_text_range() call with default params will be implicitly redirected to get_text_bounded()")


In [8]:
loader = PyMuPDFLoader(
    "../data/Retrieval Augmented Generation for Knowledge Intensive NLP Tasks.pdf"
)
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")
print(docs[0].page_content[:300])

문서의 페이지수: 19
Retrieval-Augmented Generation for
Knowledge-Intensive NLP Tasks
Patrick Lewis†‡, Ethan Perez⋆,
Aleksandra Piktus†, Fabio Petroni†, Vladimir Karpukhin†, Naman Goyal†, Heinrich Küttler†,
Mike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, Sebastian Riedel†‡, Douwe Kiela†
†Facebook AI Research; ‡University 


In [9]:
loader = PDFMinerLoader(
    "../data/Retrieval Augmented Generation for Knowledge Intensive NLP Tasks.pdf"
)
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")
print(docs[0].page_content[:300])

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

문서의 페이지수: 1
1
2
0
2

r
p
A
2
1

]
L
C
.
s
c
[

4
v
1
0
4
1
1
.
5
0
0
2
:
v
i
X
r
a

Retrieval-Augmented Generation for
Knowledge-Intensive NLP Tasks

Patrick Lewis†‡, Ethan Perez(cid:63),

Aleksandra Piktus†, Fabio Petroni†, Vladimir Karpukhin†, Naman Goyal†, Heinrich Küttler†,

Mike Lewis†, Wen-tau Yih†, Tim R


# 텍스트 분할 체크

In [10]:
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader(
    "../data/Retrieval Augmented Generation for Knowledge Intensive NLP Tasks.pdf"
)
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 19


In [11]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 158


In [12]:
print(split_documents[0].page_content)

Retrieval-Augmented Generation for
Knowledge-Intensive NLP Tasks
Patrick Lewis†‡, Ethan Perez⋆,
Aleksandra Piktus†, Fabio Petroni†, Vladimir Karpukhin†, Naman Goyal†, Heinrich Küttler†,
Mike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, Sebastian Riedel†‡, Douwe Kiela†
†Facebook AI Research; ‡University College London; ⋆New York University;
plewis@fb.com
Abstract
Large pre-trained language models have been shown to store factual knowledge


In [13]:
print(split_documents[1].page_content)

in their parameters, and achieve state-of-the-art results when ﬁne-tuned on down-
stream NLP tasks. However, their ability to access and precisely manipulate knowl-
edge is still limited, and hence on knowledge-intensive tasks, their performance
lags behind task-speciﬁc architectures. Additionally, providing provenance for their
decisions and updating their world knowledge remain open research problems. Pre-
trained models with a differentiable access mechanism to explicit non-parametric


In [14]:
# 시멘틱 청커(SemanticChunker)
# SemanticChunker를 사용하여 텍스트를 의미적으로 관련된 청크로 분할
from langchain_experimental.text_splitter import SemanticChunker

loader = PyMuPDFLoader(
    "../data/Retrieval Augmented Generation for Knowledge Intensive NLP Tasks.pdf"
)
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

docs_str = "".join([doc.page_content for doc in docs])

text_splitter = SemanticChunker(
    # OpenAI의 임베딩 모델을 사용하여 시맨틱 청커를 초기화합니다.
    OpenAIEmbeddings(),
    # 분할 기준점 유형을 백분위수로 설정합니다.
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=70,
)

# text_splitter를 사용하여 분할합니다.
split_documents = text_splitter.create_documents([docs_str])
print(f"분할된 청크의수: {len(split_documents)}")

문서의 페이지수: 19
분할된 청크의수: 211


In [15]:
print(split_documents[0].page_content)

Retrieval-Augmented Generation for
Knowledge-Intensive NLP Tasks
Patrick Lewis†‡, Ethan Perez⋆,
Aleksandra Piktus†, Fabio Petroni†, Vladimir Karpukhin†, Naman Goyal†, Heinrich Küttler†,
Mike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, Sebastian Riedel†‡, Douwe Kiela†
†Facebook AI Research; ‡University College London; ⋆New York University;
plewis@fb.com
Abstract
Large pre-trained language models have been shown to store factual knowledge
in their parameters, and achieve state-of-the-art results when ﬁne-tuned on down-
stream NLP tasks. However, their ability to access and precisely manipulate knowl-
edge is still limited, and hence on knowledge-intensive tasks, their performance
lags behind task-speciﬁc architectures.


In [16]:
print(split_documents[1].page_content)

Additionally, providing provenance for their
decisions and updating their world knowledge remain open research problems. Pre-
trained models with a differentiable access mechanism to explicit non-parametric
memory have so far been only investigated for extractive downstream tasks. We
explore a general-purpose ﬁne-tuning recipe for retrieval-augmented generation
(RAG) — models which combine pre-trained parametric and non-parametric mem-
ory for language generation. We introduce RAG models where the parametric
memory is a pre-trained seq2seq model and the non-parametric memory is a dense
vector index of Wikipedia, accessed with a pre-trained neural retriever. We com-
pare two RAG formulations, one which conditions on the same retrieved passages
across the whole generated sequence, and another which can use different passages
per token. We ﬁne-tune and evaluate our models on a wide range of knowledge-
intensive NLP tasks and set the state of the art on three open domain QA tasks,
outperfo

# docling 사용

https://python.langchain.com/docs/integrations/document_loaders/docling/

In [17]:
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType
from langchain_text_splitters import MarkdownHeaderTextSplitter


FILE_PATH = (
    "../data/Retrieval Augmented Generation for Knowledge Intensive NLP Tasks.pdf"
)

loader = DoclingLoader(file_path=FILE_PATH, export_type=ExportType.MARKDOWN)
docs = loader.load()

print(len(docs))

/Users/sean/lagom/paper-study-agent/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/sean/lagom/paper-study-agent/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


1


In [18]:
len(docs)

1

In [19]:
print(docs[0].page_content[:1000])

## Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks

Patrick Lewis †‡ , Ethan Perez glyph[star] ,

Aleksandra Piktus , Fabio Petroni , Vladimir Karpukhin , Naman Goyal , Heinrich Küttler , † † † † †

Mike Lewis , Wen-tau Yih , Tim Rocktäschel † † †‡ , Sebastian Riedel †‡ , Douwe Kiela †

† Facebook AI Research; ‡ University College London; glyph[star] New York University; plewis@fb.com

## Abstract

Large pre-trained language models have been shown to store factual knowledge in their parameters, and achieve state-of-the-art results when fine-tuned on downstream NLP tasks. However, their ability to access and precisely manipulate knowledge is still limited, and hence on knowledge-intensive tasks, their performance lags behind task-specific architectures. Additionally, providing provenance for their decisions and updating their world knowledge remain open research problems. Pretrained models with a differentiable access mechanism to explicit non-parametric memory have so 

In [20]:
splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[
        ("#", "Header_1"),
        ("##", "Header_2"),
        ("###", "Header_3"),
    ],
)
split_documents = [
    split for doc in docs for split in splitter.split_text(doc.page_content)
]
print(len(split_documents))

33


In [21]:
print(split_documents[0])

page_content='Patrick Lewis †‡ , Ethan Perez glyph[star] ,  
Aleksandra Piktus , Fabio Petroni , Vladimir Karpukhin , Naman Goyal , Heinrich Küttler , † † † † †  
Mike Lewis , Wen-tau Yih , Tim Rocktäschel † † †‡ , Sebastian Riedel †‡ , Douwe Kiela †  
† Facebook AI Research; ‡ University College London; glyph[star] New York University; plewis@fb.com' metadata={'Header_2': 'Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks'}


In [22]:
print(split_documents[1])

page_content='Large pre-trained language models have been shown to store factual knowledge in their parameters, and achieve state-of-the-art results when fine-tuned on downstream NLP tasks. However, their ability to access and precisely manipulate knowledge is still limited, and hence on knowledge-intensive tasks, their performance lags behind task-specific architectures. Additionally, providing provenance for their decisions and updating their world knowledge remain open research problems. Pretrained models with a differentiable access mechanism to explicit non-parametric memory have so far been only investigated for extractive downstream tasks. We explore a general-purpose fine-tuning recipe for retrieval-augmented generation (RAG) - models which combine pre-trained parametric and non-parametric memory for language generation. We introduce RAG models where the parametric memory is a pre-trained seq2seq model and the non-parametric memory is a dense vector index of Wikipedia, accessed

In [23]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

# # 단계 4: DB 생성(Create DB) 및 저장
# vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

# vectorstore

In [24]:
# # 단계 5: 검색기(Retriever) 생성
# retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

# retriever

In [25]:
# question = "이 논문의 저자가 누구야?"

# # retriever.invoke(question)

# for doc in retriever.stream(question):
#     print(doc.page_content)

## 이슈

FAISS 벡터 DB를 사용한 경우, 커널이 닫혀버림

해결: 다른 벡터 DB를 사용함

In [28]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone()

index_name = "paper"

if index_name not in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="dotproduct",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    print(f"{index_name} has been successfully created")
else:
    print(f"{index_name} is already exists.")

vectorstore = PineconeVectorStore.from_documents(
    split_documents, embeddings, index_name=index_name
)
retriever = vectorstore.as_retriever()

paper has been successfully created


In [29]:
question = "이 논문의 저자가 누구야?"

retriever.invoke(question)

[Document(id='bd0baffa-0a78-4835-9bc7-e1f5cfc2558c', metadata={'Header_2': 'Acknowledgments'}, page_content='The authors would like to thank the reviewers for their thoughtful and constructive feedback on this paper, as well as HuggingFace for their help in open-sourcing code to run RAG models. The authors would also like to thank Kyunghyun Cho and Sewon Min for productive discussions and advice. EP thanks supports from the NSF Graduate Research Fellowship. PL is supported by the FAIR PhD program.'),
 Document(id='14c0fa9d-9f5e-400f-ad86-499944e33d11', metadata={'Header_2': 'Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks'}, page_content='Patrick Lewis †‡ , Ethan Perez glyph[star] ,  \nAleksandra Piktus , Fabio Petroni , Vladimir Karpukhin , Naman Goyal , Heinrich Küttler , † † † † †  \nMike Lewis , Wen-tau Yih , Tim Rocktäschel † † †‡ , Sebastian Riedel †‡ , Douwe Kiela †  \n† Facebook AI Research; ‡ University College London; glyph[star] New York University; plewis@f

In [31]:
results = retriever.invoke(question)

print(results[1].page_content)

Patrick Lewis †‡ , Ethan Perez glyph[star] ,  
Aleksandra Piktus , Fabio Petroni , Vladimir Karpukhin , Naman Goyal , Heinrich Küttler , † † † † †  
Mike Lewis , Wen-tau Yih , Tim Rocktäschel † † †‡ , Sebastian Riedel †‡ , Douwe Kiela †  
† Facebook AI Research; ‡ University College London; glyph[star] New York University; plewis@fb.com


원하는데로 검색을 해오는것을 확인할수 있음

# 정리

In [34]:
# 필요한 라이브러리 임포트
from dotenv import load_dotenv
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain_teddynote import logging
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType

from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec

import warnings

warnings.filterwarnings(action="ignore")

# 환경변수 로드 (API Key 등)
load_dotenv()

# LangSmith 프로젝트 로깅 설정
logging.langsmith(project_name="Paper-Agent")

# PDF 파일 경로 지정
FILE_PATH = (
    "../data/Retrieval Augmented Generation for Knowledge Intensive NLP Tasks.pdf"
)

# 1. 문서 로드: PDF를 Markdown 형식으로 변환하여 로딩
loader = DoclingLoader(file_path=FILE_PATH, export_type=ExportType.MARKDOWN)
docs = loader.load()
print(f"문서 페이지수: {len(docs)}")

# 2. 문서 분할: Markdown 헤더 기준으로 문서 분할
splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[
        ("#", "Header_1"),
        ("##", "Header_2"),
        ("###", "Header_3"),
    ],
)
split_documents = [
    split for doc in docs for split in splitter.split_text(doc.page_content)
]
print(f"chunk 개수: {len(split_documents)}")

# 3. 임베딩 모델 생성: OpenAI 임베딩 모델 사용
embeddings = OpenAIEmbeddings()

# 4. Pinecone 인덱스 설정 및 생성 (없으면 새로 생성)
pc = Pinecone()
index_name = "paper"
if index_name not in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=1536,  # OpenAI 임베딩 벡터 차원
        metric="dotproduct",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    print(f"{index_name} 인덱스가 새로 생성되었습니다.")
else:
    print(f"{index_name} 인덱스가 이미 존재합니다.")

# 5. 분할된 문서들을 Pinecone 벡터스토어에 저장
vectorstore = PineconeVectorStore.from_documents(
    split_documents, embeddings, index_name=index_name
)
retriever = vectorstore.as_retriever()  # 검색용 래퍼

# 6. 프롬프트 템플릿 생성 (질문-응답용, 한국어 답변)
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

# 7. 언어모델(LLM) 생성: OpenAI GPT-4o 사용, 창의성 낮게(temperature=0)
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# 8. 체인(Chain) 생성: 검색 → 프롬프트 → LLM → 결과 파싱
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 9. 체인 실행: 예시 질문으로 논문 저자 확인
question = "이 논문의 저자가 누구야?"
response = chain.invoke(question)
print(response)

LangSmith 추적을 시작합니다.
[프로젝트명]
Paper-Agent
문서 페이지수: 1
chunk 개수: 33
paper 인덱스가 이미 존재합니다.
이 논문의 저자는 Patrick Lewis, Ethan Perez, Aleksandra Piktus, Fabio Petroni, Vladimir Karpukhin, Naman Goyal, Heinrich Küttler, Mike Lewis, Wen-tau Yih, Tim Rocktäschel, Sebastian Riedel, Douwe Kiela입니다.


In [39]:
# 체인 실행(Run Chain)
question = "이 논문의 Abstract 설명해줘."
response = chain.invoke(question)
print(response)

모르겠습니다.
